In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

import gym
import d4rl

from rlkit.torch.networks import Mlp
import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams['figure.figsize'] = (5, 5)

No module named 'flow'
(65544) b'X11: The DISPLAY environment variable is missing'
No module named 'carla'


In [2]:
def get_bonus(obs, actions, network, target_network):
    data = torch.cat((obs, actions), dim=1)
    # print('size of data:{}'.format(data.size()))
    
    with torch.no_grad():
        bonus = abs(network(data) - target_network(data))
    # print('size of bonus:{}'.format(bonus.size()))
    
    return bonus

In [3]:
def get_bonus_random_actions(obs, actions, network, target_network):
    action_shape = actions.shape[0]
    obs_shape = obs.shape[0]
    num_repeat = int (action_shape / obs_shape)
    obs_temp = obs.unsqueeze(1).repeat(1, num_repeat, 1).view(obs.shape[0] * num_repeat, obs.shape[1])
    # feed to the bonus networks
    data = torch.cat((obs_temp, actions), dim=1)
    with torch.no_grad():
        bonus = abs(network(data) - target_network(data))
    # reshape
    bonus = bonus.view(obs.shape[0], num_repeat, 1)
    return bonus

In [4]:
def load_models(model_name):
    
    M = 64
    network = Mlp(
        input_size=obs_dim + action_dim,
        output_size=1,
        hidden_sizes=[M, M],
    )

    target_network = Mlp(
    input_size=obs_dim + action_dim,
    output_size=1,
    hidden_sizes=[M, M],
    )
    
    path = '../models/{}'.format(model_name)
    checkpoint = torch.load(path, map_location='cpu')
    network.load_state_dict(checkpoint['network_state_dict'])
    target_network.load_state_dict(checkpoint['target_state_dict'])

    print('Loading model: {}'.format(path))
    
    return network, target_network

In [5]:
env_name = 'halfcheetah-medium-v0'

env = gym.make(env_name)
obs_dim = env.observation_space.low.size
action_dim = env.action_space.low.size

print(obs_dim)
print(action_dim)

17
6


/usr/local/google/home/shideh/virtual_env/ML/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
ds1 = d4rl.qlearning_dataset(env)
ds2 = env.get_dataset()

In [7]:
ds1.keys()

dict_keys(['observations', 'actions', 'next_observations', 'rewards', 'terminals'])

In [8]:
obs = ds1['observations']
actions = ds1['actions']
next_obs = ds1['next_observations']
terminals = ds1['terminals']

print(obs.shape)
print(next_obs.shape)

(998999, 17)
(998999, 17)


In [9]:
obs2 = ds2['observations']
actions2 = ds2['actions']
terminals2 = ds2['terminals']
time_out = ds2['timeouts']
print(obs2.shape)

(1000000, 17)


In [10]:
ind_timout= np.where(time_out==True)[0]
# print(terminals2[ind_timout[1]])

# # print(terminals2[ind_timout[1]])
# print(actions2[ind_timout[1]])
# print(actions2[ind_timout[1]+1])

In [73]:
ind = ind_timout[0] -1
print('------dqn dataset--------')
print(obs[ind,:])
print('--------------')
print(next_obs[ind,:])
print('------normal dataset--------')
print(obs2[ind,:])
print('--------------')
print(obs2[ind+1,:])


------dqn dataset--------
[ -0.30389348   0.1398849   -0.07737421   0.5346276   -0.4592523
  -0.6209193   -0.5318672   -0.3586875    2.6149719    1.5751774
  -0.64966756 -26.310863    11.040345     0.51380485   7.68483
  -2.648935     2.3205364 ]
--------------
[-0.2183516   0.04446722 -0.574744    0.74301416  0.261671    0.24509293
 -0.41752967 -0.5010071   5.2002277   2.2979493  -1.0150564   3.2571762
 -0.6951445  16.227499   16.075888    2.9142966  -7.2048073 ]
------normal dataset--------
[ -0.30389348   0.1398849   -0.07737421   0.5346276   -0.4592523
  -0.6209193   -0.5318672   -0.3586875    2.6149719    1.5751774
  -0.64966756 -26.310863    11.040345     0.51380485   7.68483
  -2.648935     2.3205364 ]
--------------
[-0.2183516   0.04446722 -0.574744    0.74301416  0.261671    0.24509293
 -0.41752967 -0.5010071   5.2002277   2.2979493  -1.0150564   3.2571762
 -0.6951445  16.227499   16.075888    2.9142966  -7.2048073 ]


In [74]:
ind = ind_timout[0] 
print('------dqn dataset--------')
print(obs[ind,:])
print('--------------')
print(next_obs[ind,:])
print('------normal dataset--------')
print(obs2[ind,:])
print('--------------')
print(obs2[ind+1,:])

------dqn dataset--------
[ 0.03488301  0.099709    0.01332847 -0.08567016 -0.09784969 -0.09340785
  0.07143071  0.05414234  0.30647758  0.03224927 -0.01242412  0.1309806
 -0.0541737   0.02604712  0.20787422  0.02981176 -0.00630953]
--------------
[ 0.01895389  0.11537578  0.30336612 -0.40792343  0.305279    0.24464652
 -0.2839574  -0.22878769  1.1908333  -0.68324995  0.13777132  7.047706
 -6.767168    9.79017     8.537244   -7.996458   -7.8005657 ]
------normal dataset--------
[-0.2183516   0.04446722 -0.574744    0.74301416  0.261671    0.24509293
 -0.41752967 -0.5010071   5.2002277   2.2979493  -1.0150564   3.2571762
 -0.6951445  16.227499   16.075888    2.9142966  -7.2048073 ]
--------------
[ 0.03488301  0.099709    0.01332847 -0.08567016 -0.09784969 -0.09340785
  0.07143071  0.05414234  0.30647758  0.03224927 -0.01242412  0.1309806
 -0.0541737   0.02604712  0.20787422  0.02981176 -0.00630953]


In [35]:
terminal_inds = np.where(terminals==True)
print(terminal_inds)

(array([898]),)


In [37]:
terminals[898+100]

False

In [38]:
ds3 = d4rl.qlearning_dataset(env, ds2)


In [39]:
obs = ds3['observations']
actions = ds3['actions']
next_obs = ds3['next_observations']
terminals = ds3['terminals']

print(obs.shape)
print(next_obs.shape)

(998999, 17)
(998999, 17)


 So, for a transition $(s_t,a_t,r_t,s_{t+1})$, the minimum can reasonably be approximated by
min_{a} b(s_t, a) = b(s_t, a_t)

In [ ]:
o